In [3]:
import os
import pandas as pd
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent, AgentChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents.strategies import DefaultTerminationStrategy, KernelFunctionSelectionStrategy, KernelFunctionTerminationStrategy

from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.connectors.ai import PromptExecutionSettings
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from semantic_kernel.functions import KernelPlugin, KernelFunctionFromPrompt
from azure.ai.evaluation import SimilarityEvaluator
from crm_store import CRMStore
from product_store import ProductStore
import agentic_system

load_dotenv(override=True)

None


ValueError: Invalid URL scheme or hostname: SplitResultBytes(scheme=b'', netloc=b'', path=b'', query=b'', fragment=b'')

In [2]:
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = "2024-06-01"

API_BASE = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}"

print(AZURE_OPENAI_DEPLOYMENT_NAME)

gpt-4o-2024-05-13


In [3]:
# Initialize Azure OpenAI Connection with your environment variables
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

similarity = SimilarityEvaluator(model_config)

In [ ]:
import time
def get_agentic_response(row):
    query = row['Queries']
    ground_truth = row['Answer']
    
    first_column_name = row.index[0]
    first_field = row[first_column_name]
    
    res = agentic_system.invoke(query=query)
    
    print(f"Record : {first_field} - {query} - {res.get('count')}")
    
    content = res.get("content", str(res))
    name = res.get("name", "ERROR")
    similarity_score = similarity(query=query, response=content, ground_truth=ground_truth)
    
    time.sleep(1)
    # Return the content and name as a Series
    return pd.Series({'agentic_answer': content, 'agent_name': name, 'similarity_score': similarity_score.get('gpt_similarity')})

In [ ]:
df = pd.read_csv('../../../../travel_question_bank_answered_anon.csv')

In [ ]:
# SINGLE RECORD
row = df.iloc[0]

print(f"ROW:\n\n{row}\n\n")

dd = get_agentic_response(row)

print(f"Agent name: {dd['agent_name']}")
print(f"Similarity score: {dd['similarity_score']}")
print(f"Agentic answer: {dd['agentic_answer']}")
print(f"---------------------------")
print(f"Ground Truth Answer: {row['Answer']}")

In [ ]:
df[['agentic_answer', 'agent_name', 'similarity_score']] = df.apply(get_agentic_response, axis=1)

In [8]:
df.groupby('agent_name')['similarity_score'].mean().sort_values(ascending=False)

agent_name
QUERY      3.254237
PLANNER    1.000000
Name: similarity_score, dtype: float64

In [9]:
df.groupby('agent_name')['similarity_score'].count().sort_values(ascending=False)

agent_name
QUERY      59
PLANNER     1
Name: similarity_score, dtype: int64

In [ ]:
mean_similarity_score = df['similarity_score'].mean()
print(f"The mean similarity score is: {mean_similarity_score:.2f}")

The mean similarity score is: 3.22


In [ ]:
df.to_csv('agentic_responses_sk.csv', index=False)